# EE 304 - Neuromorphics: Brains in Silicon


##  Quadratic Integrate & Fire Neuron: Silicon Circuit

<img src="files/lecture9/QIF-ckt-vmem.png" width="360">

The story thus far:

- We learned how to use phase-plots to understand nonlinear dynamical systems
    - $\dot{v}$ is plotted against $v$
    - All trajectories lie on this line
    - Fixed points occur where it crosses $\dot{v}=0$
- We analyzed the quadratic integrate-and-fire (QIF) model
    - It is the simplest spike-generating model
    - It transitions from quiescence to spiking through a saddle-node bifurcation
- We derived the QIF model's frequency-current (FI) curve 
    - Its spike-rate increases as the square-root of its input current

####  Outline for this lecture

We will design a silicon circuit that implements the QIF model

- We will analyze our circuit design to understand the impact of transistor mismatch
- We will find that the input current is scaled by a lognormally distributed gain 
- And the minimum current required for spiking is also lognormally distributed 

### Subtheshold MOS QIF Implementation: $I_C(I_u,I_v)$

<img src="files/lecture9/generic-lpf-ckt.png" width="360">

To implement the QIF model in silicon, we start with its dimensionless differential equation:

$$
    \tau \frac{dv}{dt} = - v + \frac{1}{2}v^2 + u
$$

We make the substitutions $v = I_v/I_{\rm n}$ and $u = I_u/I_{\rm n}$, where $I_{\rm n}$ is the unit current:

\begin{eqnarray*}
    \tau\frac{d}{dt}\left(\frac{I_v}{I_{\rm n}}\right) & = & \
    -\frac{I_v}{I_{\rm n}} + \frac{1}{2}\left(\frac{I_v}{I_{\rm n}}\right)^2 + \frac{I_u}{I_{\rm n}} \\
    \frac{\tau}{I_{\rm n}}\frac{dI_v}{dV_v}\frac{dV_v}{dt} & = & \
    -\frac{I_v}{I_{\rm n}} + \frac{1}{2}\left(\frac{I_v}{I_{\rm n}}\right)^2 + \frac{I_u}{I_{\rm n}} 
\end{eqnarray*}

Substituting $dI_v/dV_v=\kappa I_v/U_T$ and multiplying both sides by $I_{\rm n}/I_v$ yields:

$$
    \frac{\kappa\tau}{U_T}\frac{dV_v}{dt} \
    = -1 + \frac{1}{2}\frac{I_v}{I_{\rm n}} + \frac{I_u}{I_v} 
$$

- which yields 

$$
    C\frac{dV_v}{dt} \
    = -I_{\rm lk} + \frac{1}{2}\frac{I_{\rm lk}}{I_{\rm n}}I_v + \frac{I_{\rm lk}}{I_v}I_u 
$$

- where $I_{\rm lk} = C U_T /\kappa\tau$

### Subtheshold MOS QIF Implementation: Circuit Design 

<img src="files/lecture9/QIF-full-ckt.png" width="640">

We derived the capacitor's current:  

$$
    C\frac{dV_v}{dt} \
    = -I_{\rm lk} + \frac{1}{2}\frac{I_{\rm lk}}{I_{\rm n}}I_v + \frac{I_{\rm lk}}{I_v}I_u 
$$

- the second term--linearly proportional to $I_v$--implements the quadratic term  

If we choose $I_{\rm n} = I_{\rm lk}/2$, we obtain: 

$$
    C\frac{dV_v}{dt} \
    = -I_{\rm lk} + I_v + \frac{I_{\rm lk}}{I_v}I_u 
$$

- hence, all we have to do is to copy $I_v$ back to $V_v$
- we accomplish this with a current mirror (see figure)
 
Our choice of $I_{\rm n}$ implies that:

- $I_v = I_{\rm lk}/2$ at threshold
- $I_u = I_{\rm lk}/4$ is the minimum current required to spike
    - where $I_{\rm lk} = C U_T /\kappa\tau$

### Subtheshold MOS QIF Implementation: Including Mismatch 

<img src="files/lecture9/QIF-mismatch-ckt.png" width="640">

We now analyze our QIF circuit design, accounting for the transistors' current gains.

This analysis yields:
$$
    I_3 = \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}\frac{I_u I_{\rm lk}}{I_v}
    \;\; {\rm and} \;\;
    I_{\rm fb} =\frac{\Lambda_6}{\Lambda_5}I_v
$$

Equating the rate of change of the capacitor's charge to the net current supplied, we have 

\begin{eqnarray*}
    C\frac{dV_v}{dI_v}\frac{dI_v}{dt} & = & 
    - I_{\rm lk} + \frac{\Lambda_6}{\Lambda_5}I_v + \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}\frac{I_u I_{\rm lk}}{I_v}\\
    \iff \frac{C U_T}{\kappa I_v}\frac{dI_v}{dt} & = & 
    - I_{\rm lk} + \frac{\Lambda_6}{\Lambda_5}I_v + \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}\frac{I_u I_{\rm lk}}{I_v} \\
    \iff \frac{C U_T}{\kappa I_{\rm lk}}\frac{dI_v}{dt} & = & - I_v + \frac{\Lambda_6}{\Lambda_5}\frac{I_v^2}{I_{\rm lk}} + \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}I_u 
\end{eqnarray*}

Converting to a dimensionless model by normalizing with $I_{\rm n}=\frac{1}{2}I_{\rm lk}$ yields

$$
     \underbrace{ \frac{C U_T}{\kappa I_{\rm lk}} }_\tau 
     \underbrace{ \frac{d}{dt}\frac{I_v}{\frac{1}{2}I_{\rm lk}} }_\dot{v}v =
       - \underbrace{ \frac{I_v}{\frac{1}{2}I_{\rm lk}} }_v
       + \frac{\Lambda_6}{\Lambda_5}\frac{1}{2}
         \underbrace{ \left(\!\!\frac{I_v}{\frac{1}{2}I_{\rm lk}}\!\!\right)^{\! 2} }_{v^2}
       + \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}\underbrace{ \frac{I_u}{\frac{1}{2}I_{\rm lk}} }_u 
$$

### Subtheshold MOS QIF Implementation: Effect of Mismatch 

<img src="files/lecture9/mismatch-response-curve.png" width="1200">

With mismatch included, our circuit implements the following dimensionless QIF model:

$$
     \tau\frac{dv}{dt} = 
     - v + \frac{\Lambda_6}{\Lambda_5}\frac{1}{2}v^2 + \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}u 
$$
- $\Lambda_1, \dots, \Lambda_6$ are lognormally distributed
- Their underlying normal distribution has mean 0 and standard deviation $\sigma_{U_T}$ (dimensionless)

Thus, each instance of this model has a different input gain, $A_{\rm in}=\Lambda_3\Lambda_4/\Lambda_1\Lambda_2$:

- It is also lognormally distributed
- Its underlying normal distribution has standard deviation $2\sigma_{U_T}$

What impact does mismatch have on the model's threshold voltage and threshold current?

Recall that a saddle-node bifurcation occurs in the dynamical system $\dot{v} = f(v, u)$ when:

- $f(v_{\rm th}, u_{\rm th}) = 0$ and $f_v(v_{\rm th}, u_{\rm th}) = 0$

For the QIF model with mismatch, we have

$$ 
\left\{ 
  \begin{matrix}
    - v_{\rm th} + \frac{\Lambda_6}{\Lambda_5}\frac{1}{2}v_{\rm th}^2 
    + \frac{\Lambda_3\Lambda_4}{\Lambda_1\Lambda_2}u_{\rm th} & = & 0\\
    - 1 +  \frac{\Lambda_6}{\Lambda_5}v_{\rm th} & = & 0 
  \end{matrix} \right.
\implies 
\left\{ 
  \begin{array}{l l}
     u_{\rm th} = \frac{\Lambda_1\Lambda_2\Lambda_5}{\Lambda_3\Lambda_4\Lambda_6}\frac{1}{2}\\
     v_{\rm th} = \frac{\Lambda_5}{\Lambda_6}
  \end{array} \right.
$$

Thus, the threshold voltage and threshold current are both lognormally distributed

- Their underlying normal distributions' standard deviations are $\sqrt{6}\sigma_{U_T}$ and $\sqrt{2}\sigma_{U_T}$, respectively

The curves above have $\sigma_{U_T}=0.05, 0.22, 0.40$; an offset $i_{\rm bg} = 0.7$ was added to $u$

- This model's circuit is a little more complex than the one analyzed here 
- See Homework 2